In [1]:
import time
import networkx as nx
from braket.aws import AwsDevice, Devices
from src.graphconnector import AdvancedGraphSolver
import matplotlib.pyplot as plt

max_connectivity = 4 # Max number of connections per qubit

Emerald_device = AwsDevice(Devices.IQM.Emerald)

# Solving graph problem using src/graphconnector.py

start_time = time.time()
print("Initializing solver...")

total_connectivity = Emerald_device.properties.paradigm.connectivity.connectivityGraph              # Get device connectivity
qubits = [qub for qub in total_connectivity]                                                        # List of qubits in device
qubits.sort(key=float)                                                                              # Sort qubits numerically

solver = AdvancedGraphSolver(total_connectivity)                                                    # Initialize solver class

print("Phase 1: Generating valid partition (Kempe Chains)...")
initial_sets = solver.color_initial_edges()

print("Phase 2: Maximizing density (Augmenting Paths)...")
final_sets = solver.maximize_sets(initial_sets)

end_time = time.time()

print(f"\nDONE in {end_time - start_time:.4f} seconds.")
print(f"Total Edges in Graph: {len(solver.all_edges)}")
print("-" * 30)

for i, s in enumerate(final_sets):
    # Sort for readable output
    sorted_s = sorted(list(s), key=lambda x: int(x[0]))
    print(f"Set {i+1} ({len(s)} pairs): {sorted_s}")
    
    # Verification
    nodes = [n for p in s for n in p]
    if len(nodes) != len(set(nodes)):
        print("  ERROR: Duplicate node detected!")


ImportError: cannot import name 'Devices' from 'braket.aws' (/Users/chardiol/Desktop/Pawsey!/UnNoiseCompile/.venv/lib/python3.11/site-packages/braket/aws/__init__.py)

In [ ]:
# Example usage for the Emerald device

Emerald_device = AwsDevice(Devices.IQM.Emerald)

G = Emerald_device.topology_graph

for i in range(4):
    
    red_edges = list(final_sets[i])
    connected_qubits = [str(qub_pair[0]) for qub_pair in red_edges] + [str(qub_pair[1]) for qub_pair in red_edges]
    connected_qubits.sort(key=float)
    print("Connected Qubits:\n",connected_qubits,"\n")
    print("Total Qubits:\n",qubits,"\n")
    print("Efficiency:", len(connected_qubits)/len(qubits))
    
    # 3. Create a color map based on G.edges()
    edge_colors = []
    for u, v in G.edges():
        u = str(u)
        v = str(v)
        # Check if the edge (or its reverse) is in your red_subset
        if (u, v) in red_edges or (v, u) in red_edges:
            edge_colors.append('red')
        else:
            edge_colors.append('black')
    
    black_edges = [e for e in G.edges() if e not in red_edges and (e[1], e[0]) not in red_edges]
    
    nx.draw_kamada_kawai(Emerald_device.topology_graph, with_labels=True,edge_color=edge_colors, font_color="white")
    plt.show()